In [ ]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Model
import faiss
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.cluster import normalized_mutual_info_score

In [ ]:
tf.keras.backend.clear_session()

test_model = tf.keras.models.load_model('./test_model', compile=False)

In [ ]:
def Actual_and_predict(final_image):

    
    image = tf.io.read_file(final_image)
    
    image = tf.image.decode_jpeg(image, channels=3)
    
    image = tf.image.resize(image, [64,64])
    
    image = tf.cast(image, tf.float32)
    
    image /= 255.0
    
    test_embeddings = test_model(np.expand_dims(image,0))
    
    norm = np.linalg.norm(test_embeddings, axis=1, keepdims=True)
    test_embeddings = test_embeddingss / norm
    
    test_embeddings = test_embeddings.numpy()
    
    
    index = faiss.read_index("../input/k-means-trained/kmeans_trained.index")
    
    distances, indices = index.search(test_embeddingss,5)
    
    with open('./labelencoder.pkl', "rb") as input_file:
        le = pickle.load(input_file)
    
    actual_output_labels = le.inverse_transform(indices[0])
    
    return actual_output_labels

In [ ]:
def predict_error(final_image,actual_y):
    
    pred_labels = Actual_and_predict(final_image)
    
    pred_labels = pred_labels.squeeze()
    
    score = normalized_mutual_info_score(actual_y, np.array([pred_labels[0]]))
    
    with open("../input/k-means-error/kmeans_error.pkl", "rb") as f:
        total_error = pickle.load(f)
    
    
    return total_error, score

In [ ]:
final_image = '../input/final-image/8007fa0a8fe89f9d.jpg'

actual_y = np.array([63866])

In [ ]:
Actual_and_predict(final_image)

array([21020, 58967,  4660, 32857, 42637])

In [ ]:
predict_error(final_image,actual_y)

(21610.96484375, 1.0)